<a href="https://colab.research.google.com/github/callumselv/Y4_project/blob/main/Lohani_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 32.5 MB/s eta 0:00:00


In [20]:
import numpy as np
from qutip import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

In [4]:
from scipy.stats import unitary_group

In [23]:
no_data = 10000
d = 2
dimension = 2**d

In [24]:
# Generate rho matrices

rhos = np.zeros((no_data,dimension,dimension),dtype=complex)
eta = 1e-7

for i in range(no_data):
    random_unitary = unitary_group.rvs(dimension)
    coeffs = random_unitary[:,0]
    rho = np.outer(coeffs,coeffs.conj())
    rhos[i] = (1-eta)*rho + (eta/4)*np.eye(dimension)

In [11]:
cholesky_decompositions = np.array([np.linalg.cholesky(rho) for rho in rhos])

In [14]:
all_taus = np.zeros((no_data, dimension**2))
for no in range(no_data):
    for cholesky in cholesky_decompositions:
        extracted_values = []
        for i in range(cholesky.shape[0]):
            extracted_values.append(cholesky[i, i].real)

        primary_indices = [(1, 0), (2, 1), (3, 2)]
        for i, j in primary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)

        secondary_indices = [(2, 0), (3, 1), (3, 0)]
        for i, j in secondary_indices:
            extracted_values.append(cholesky[i, j].real)
            extracted_values.append(cholesky[i, j].imag)
    all_taus[no] = np.array(extracted_values)


In [21]:
tau_vector = extracted_values

In [15]:
def reconstruct_matrix(taus):
    matrix = np.zeros((4, 4), dtype=np.complex_)

    index = 0
    for i in range(4):
        matrix[i, i] = taus[index]
        index += 1

    primary_indices = [(1, 0), (2, 1), (3, 2)]
    for i, j in primary_indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    secondary_indices = [(2, 0), (3, 1), (3, 0)]
    for i, j in secondary_indices:
        real_part = taus[index]
        imag_part = taus[index + 1]
        matrix[i, j] = real_part + 1j * imag_part
        index += 2

    return matrix

In [16]:
# Make 6^d projectors of Pauli {X,Y,Z} operators

paulis = [sigmax(),sigmay(),sigmaz()]
evectors = np.zeros((6,2,1),dtype=complex)

i=0
for pauli in paulis:
    vals, vecs = pauli.eigenstates()
    evectors[i] = vecs[0][:]
    evectors[i+1] = vecs[1][:]
    i += 2

In [17]:
projectors = np.zeros((6,2,2), dtype=complex)

for i, vectors in enumerate(evectors):
    projectors[i] = np.outer(vectors,vectors.conj())

In [18]:
pauli_2d = np.zeros((6**2,4,4), dtype=complex)

index=0
for i in projectors:
    for j in projectors:
        pauli_2d[index] = np.kron(i,j)
        index += 1

In [19]:
nvals = np.zeros(6**2)

for index,proj in enumerate(pauli_2d):
    nvals[index] = np.trace(rhos[0]@proj)

<ipython-input-19-d4a2b74120f7>:4: ComplexWarning: Casting complex values to real discards the imaginary part
  nvals[index] = np.trace(rhos[0]@proj)


In [22]:
def build_cnn_model():
    model = Sequential()

    model.add(tf.keras.layers.Reshape((6, 6, 1), input_shape=(36,)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=25, kernel_size=(2, 2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(750, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(450, activation='relu'))
    model.add(Dropout(0.5))

    tau_dim = len(tau_vector)
    model.add(Dense(tau_dim, activation='linear'))

    return model

model = build_cnn_model()

model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.005),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 6, 6, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 5, 5, 25)            │             125 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 2, 2, 25)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 1, 1, 25)            │           2,525 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 0, 0, 25)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 0)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 750)                 │             750 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 750)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 450)                 │         337,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 450)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │           7,216 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 348,566 (1.33 MB)

 Trainable params: 348,566 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# `x_train`, `y_train` are probabilities (n values) and labels (tau-vectors)
model.fit(x_train, y_train, epochs=300, batch_size=100, validation_split=0.2)